In [1]:
import scrapy

In [2]:
class Spider12(scrapy.Spider):
    name = 'Spider12'
    allowed_domains = ['pagina12.com.ar']
    custom_settings = {'FEED_FORMAT': 'json', 
                       'FEED_URI':'resultados.json',
                       'DEPTH_LIMIT': 2}
    
    start_urls = ['https://www.pagina12.com.ar/secciones/el-pais',
                  'https://www.pagina12.com.ar/secciones/economia',
                  'https://www.pagina12.com.ar/secciones/sociedad',
                  'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                  'https://www.pagina12.com.ar/secciones/ciencia',
                  'https://www.pagina12.com.ar/secciones/el-mundo',
                  'https://www.pagina12.com.ar/secciones/deportes',
                  'https://www.pagina12.com.ar/secciones/contratapa']
    
    def parse(self, response):
        # Articulo prmocionado
        nota_promocionada = response.xpath('//div[@class="featured-article__container"]/h2/a/@href').get()
        if nota_promocionada is not None:
            yield response.follow(nota_promocionada, callback=self.parse_nota)
        
        # Listado de notas
        notas = response.xpath('//ul[@class="article-list"]//li//a/@href').getall()
        for nota in notas:
            yield response.follow(nota, callback=self.parse_nota)
        
        # Link a la siguiente página
        next_page = response.xpath('//a[@class="pagination-btn-next"]/@href')
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)
            
    def parse_nota(self, response):
        # Parseo la nota
        titulo = response.xpath('//div[@class="article-title"]/text()').get()
        cuerpo = ''.join(response.xpath('//div[@class="article-text"]/p/text()').getall())
        yield {'url': response.url,
               'titulo': titulo,
               'cuerpo': cuerpo}

In [5]:
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess()
process.crawl(Spider12)
process.start()

2019-11-06 14:43:25 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-11-06 14:43:25 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.6.8 (default, Aug 20 2019, 17:12:48) - [GCC 8.3.0], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.7, Platform Linux-5.0.0-32-generic-x86_64-with-Ubuntu-18.04-bionic
2019-11-06 14:43:25 [scrapy.crawler] INFO: Overridden settings: {'DEPTH_LIMIT': 2, 'FEED_FORMAT': 'json', 'FEED_URI': 'resultados.json'}
2019-11-06 14:43:25 [scrapy.extensions.telnet] INFO: Telnet Password: 66ba7a728d842133
2019-11-06 14:43:25 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2019-11-06 14:43:25 [scrapy.middleware] INFO: Enabled downloader middlewares:

ReactorNotRestartable: 